In [1]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [2]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2024-02-01 10:05:48--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 172.67.70.149, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip’

book-crossings.zip  100%[===================>]  24.88M   160MB/s    in 0.2s    

2024-02-01 10:05:49 (160 MB/s) - ‘book-crossings.zip’ saved [26085508/26085508]

Archive:  book-crossings.zip
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [6]:
# import csv data into dataframes
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
df_books = pd.read_csv(
    'BX-Books.csv',
    encoding='ISO-8859-1',
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'}
)

df_ratings = pd.read_csv(
    'BX-Book-Ratings.csv',
    encoding='ISO-8859-1',
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'}
)



In [7]:
# Unisci i dati di libri e valutazioni
df = pd.merge(df_ratings, df_books, on='isbn')

# Creazione di un dataframe contenente le informazioni sui libri per la costruzione del modello
books_info = df.groupby('title')['rating'].count().reset_index()
books_info.columns = ['title', 'ratings_count']

# Filtro per considerare solo i libri con un certo numero di valutazioni
min_ratings_count = 50
popular_books_info = books_info[books_info['ratings_count'] >= min_ratings_count]

# Filtro il dataframe originale per considerare solo i libri popolari
df_popular_books = df[df['title'].isin(popular_books_info['title'])]

# Creazione di un modello Nearest Neighbors utilizzando il conteggio delle valutazioni
nn_model = NearestNeighbors(n_neighbors=6, algorithm='brute', metric='cosine')

# Creazione di una matrice user-item per Nearest Neighbors
user_item_matrix = df_popular_books.pivot_table(index='user', columns='title', values='rating', fill_value=0)

# Fitting del modello Nearest Neighbors
nn_model.fit(user_item_matrix.values.T)



NearestNeighbors(algorithm='brute', metric='cosine', n_neighbors=6)

In [8]:
def get_recommends(book_title):
    # Trova l'indice del libro nel dataset
    book_index = user_item_matrix.columns.get_loc(book_title)

    # Ottieni le distanze e gli indici dei libri più vicini
    distances, indices = nn_model.kneighbors(user_item_matrix.iloc[:, book_index].values.reshape(1, -1))

    # Crea una lista di libri consigliati con le relative distanze
    recommended_books = [
        [user_item_matrix.columns[i], distances[0, j]] for j, i in enumerate(indices.flatten()) if i != book_index
    ]

    # Restituisci il risultato come una lista [titolo, lista consigliati]
    return [book_title, recommended_books]

# Esempio di utilizzo
result = get_recommends("The Queen of the Damned (Vampire Chronicles (Paperback))")
print(result)

['The Queen of the Damned (Vampire Chronicles (Paperback))', [['The Tale of the Body Thief (Vampire Chronicles (Paperback))', 0.5620454125950052], ['The Vampire Lestat (Vampire Chronicles, Book II)', 0.595377238255751], ['Interview with the Vampire', 0.7202435286233742], ['Memnoch the Devil (Vampire Chronicles, No 5)', 0.7420628214446703], ['Pandora: New Tales of the Vampires (New Tales of the Vampires)', 0.8440147701469674]]]


In [9]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [['The Lovely Bones: A Novel', 0.9095981950904724], ['Blue Diary', 0.9101138470509211], ['I Know This Much Is True', 0.9102432841135182], ["The Pilot's Wife : A Novel", 0.9172570741861605], ['The Weight of Water', 0.9210187581789657]]]
You haven't passed yet. Keep trying!
